# Reset

In [3]:
-- USE master;

-- DROP DATABASE [APU Bus Services];

-- DROP CERTIFICATE TDECert_BusService;

-- DROP LOGIN AnalystDeptHead;
-- DROP LOGIN ServicesDeptHead;
-- DROP LOGIN DatabaseAdmin1;
-- DROP LOGIN Scheduler1;
-- DROP LOGIN Scheduler2;
-- DROP LOGIN Student1;
-- DROP LOGIN Student2;
-- DROP LOGIN Student3;

-- DROP TRIGGER AuditLogon ON ALL SERVER;
-- DROP TRIGGER LimitConnectionAfterOfficeHours ON ALL SERVER;
-- DROP TRIGGER LimitManagementLoginHours ON ALL SERVER;
-- DROP TRIGGER LimitSessions ON ALL SERVER;
-- DROP TRIGGER LimitStudentLoginHours ON ALL SERVER;
-- DROP TRIGGER LimitStudentSessions ON ALL SERVER;
-- DROP TRIGGER MyHostsOnly ON ALL SERVER;

Commands completed successfully.

Total execution time: 00:00:00.658

# Create Database

In [4]:
CREATE DATABASE [APU Bus Services];

Commands completed successfully.

Total execution time: 00:00:01.221

# Implement Transparent Data Encryption (TDE)

## Chew Cheng Yong TP051338

In [5]:
USE master;

CREATE CERTIFICATE TDECert_BusService 
WITH SUBJECT ='TDECert_BusService';

Commands completed successfully.

Total execution time: 00:00:00.168

In [6]:
USE [APU Bus Services];

CREATE DATABASE ENCRYPTION KEY WITH ALGORITHM = AES_256
ENCRYPTION BY SERVER CERTIFICATE TDECert_BusService;

Warning: The certificate used for encrypting the database encryption key has not been backed up. You should immediately back up the certificate and the private key associated with the certificate. If the certificate ever becomes unavailable or if you must restore or attach the database on another server, you must have backups of both the certificate and the private key or you will not be able to open the database.

Total execution time: 00:00:00.037

In [7]:
ALTER DATABASE [APU Bus Services] SET ENCRYPTION ON;

Commands completed successfully.

Total execution time: 00:00:00.015

## Wong Poh Yee TP051079

In [8]:
-- USE master;

-- CREATE ASYMMETRIC KEY APUServicesKey
-- WITH ALGORITHM = RSA_2048
-- ENCRYPTION BY PASSWORD = 'Passw0rd';

Commands completed successfully.

Total execution time: 00:00:00

In [9]:
-- USE [APU Bus Services];

-- CREATE DATABASE ENCRYPTION KEY WITH ALGORITHM = AES_256
-- ENCRYPTION BY SERVER ASYMMETRIC KEY APUServicesKey;

Commands completed successfully.

Total execution time: 00:00:00

In [10]:
-- ALTER DATABASE [APU Bus Services] SET ENCRYPTION ON;

Commands completed successfully.

Total execution time: 00:00:00.001

# Use Database

In [11]:
USE [APU Bus Services];

Commands completed successfully.

Total execution time: 00:00:00

# Create Schemas

In [12]:
CREATE SCHEMA Management;

Commands completed successfully.

Total execution time: 00:00:00.003

In [13]:
CREATE SCHEMA BusUser;

Commands completed successfully.

Total execution time: 00:00:00.001

# Create DDL Triggers

## Chew Cheng Yong TP051338

In [14]:
CREATE TABLE Management.AuditLog_DDL (
    AuditLogID INT IDENTITY(1, 1) NOT NULL,
    LogDate DATETIME DEFAULT GETDATE() NOT NULL,
    UserName SYSNAME DEFAULT USER_NAME() NOT NULL,
    SQLCmd NVARCHAR(max)
);

Commands completed successfully.

Total execution time: 00:00:02.702

In [15]:
CREATE or ALTER TRIGGER AuditChange
ON DATABASE
FOR 
    CREATE_TABLE, 
    ALTER_TABLE, 
    DROP_TABLE, 
    CREATE_VIEW, 
    ALTER_VIEW, 
    DROP_VIEW
AS 
	IF (IS_MEMBER('DatabaseAdmins') = 1)
    BEGIN
        DECLARE @SQLCmd NVARCHAR(MAX);
        SELECT @SQLCmd = EVENTDATA().value('(/EVENT_INSTANCE/TSQLCommand/CommandText)[1]', 'NVARCHAR(MAX)');
        INSERT INTO Management.AuditLog_DDL (SQLCmd) VALUES(@SQLCmd);
    END
    ELSE
    BEGIN
        PRINT 'ERROR: Current user does not have permission to CREATE/DROP/ALTER!!';
        ROLLBACK;
    END;

Commands completed successfully.

Total execution time: 00:00:02.711

In [16]:
CREATE or ALTER TRIGGER AuditChange
ON DATABASE
FOR 
    CREATE_TABLE, 
    ALTER_TABLE, 
    DROP_TABLE, 
    CREATE_VIEW, 
    ALTER_VIEW, 
    DROP_VIEW
AS 
BEGIN
    DECLARE @SQLCmd NVARCHAR(MAX);
    SELECT @SQLCmd = EVENTDATA().value('(/EVENT_INSTANCE/TSQLCommand/CommandText)[1]', 'NVARCHAR(MAX)');
    INSERT INTO Management.AuditLog_DDL (SQLCmd) VALUES(@SQLCmd);
END;

Commands completed successfully.

Total execution time: 00:00:00.009

In [17]:
--only database administrator can drop table, however, there are several tables that cannot be deleted
CREATE OR ALTER TRIGGER NoDropTable
ON DATABASE
FOR DROP_TABLE
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @Schema SYSNAME = EVENTDATA().value('(/EVENT_INSTANCE/SchemaName)[1]', 'sysname');
    DECLARE @Table SYSNAME = EVENTDATA().value('(/EVENT_INSTANCE/ObjectName)[1]', 'sysname');

    IF IS_MEMBER('DatabaseAdmins') = 1
    BEGIN
	    IF @Schema IN ('Management','BusUser') 
        AND @Table IN ('Reservation', 'BusStatus', 'Bus', 'Student', 'Route', 'Schedule', 'Station', 'TimeSlot')
			BEGIN
				PRINT 'ERROR: [' + @Schema + '].[' + @Table + '] cannot be dropped.';
				ROLLBACK;
			END
	    ELSE
			BEGIN
				INSERT INTO Management.AuditTable (
					Event_Data,
					ChangedBy,
					ChangedOn
				)
				VALUES (
					EVENTDATA(),
					USER,
					GETDATE()
				);
			END
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.006

In [18]:
CREATE OR ALTER TRIGGER NoCreate
ON DATABASE
FOR 
	CREATE_TABLE,
    CREATE_VIEW
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @Schema SYSNAME = EVENTDATA().value('(/EVENT_INSTANCE/SchemaName)[1]', 'sysname');
    IF (IS_MEMBER('DatabaseAdmins') = 1)
    BEGIN
        IF @Schema IN ('Management','BusUser') 
        BEGIN
            INSERT INTO Management.AuditTable (
                Event_Data,
                ChangedBy,
                ChangedOn
            )
            VALUES (
                EVENTDATA(),
                USER,
                GETDATE()
            );
        END
        ELSE
        BEGIN
            PRINT 'ERROR: Cannot create table/view outside '+ @Schema;
            ROLLBACK;
        END
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.003

## Choong Man Shun TP051283

In [19]:
CREATE OR ALTER TRIGGER DropTableSafety
ON DATABASE
FOR DROP_TABLE
AS
	PRINT 'You must disable trigger DropTableSafety to drop tables.';
	ROLLBACK;

Commands completed successfully.

Total execution time: 00:00:00.002

In [20]:
CREATE OR ALTER TRIGGER AlterTableSafety
ON DATABASE
FOR ALTER_TABLE
AS
	PRINT 'You must disable trigger AlterTableSafety to alter tables.';
	ROLLBACK;

Commands completed successfully.

Total execution time: 00:00:00.001

# Create Tables

## Wong Poh Yee TP051079

In [21]:
CREATE TABLE Management.Station (
	StationID INT PRIMARY KEY IDENTITY(1, 1),
	Name VARCHAR(200) UNIQUE NOT NULL,
	ShortName VARCHAR(20) UNIQUE NOT NULL
);

(1 row affected)

Total execution time: 00:00:01.278

In [22]:
CREATE TABLE Management.Route (
	RouteID INT PRIMARY KEY IDENTITY(1, 1),
	DepartureStationID INT REFERENCES Management.Station(StationID) NOT NULL,
	ArrivalStationID INT REFERENCES Management.Station(StationID) NOT NULL,
	Name VARCHAR(200) UNIQUE NOT NULL,
	CONSTRAINT Route_CK1 CHECK (DepartureStationID <> ArrivalStationID)
);

(1 row affected)

Total execution time: 00:00:00.257

In [23]:
CREATE TABLE Management.TimeSlot (
	TimeSlotID INT PRIMARY KEY IDENTITY(1, 1),
	DepartureDatetime DATETIME NOT NULL,
	ArrivalDatetime DATETIME NOT NULL,
	CONSTRAINT TimeSlot_CK1 CHECK (DepartureDatetime < ArrivalDatetime)
);

(1 row affected)

Total execution time: 00:00:02.021

## Chew Cheng Yong TP051338

In [24]:
CREATE TABLE Management.BusStatus (
	StatusID INT PRIMARY KEY IDENTITY(1, 1),
	Name VARCHAR(200) UNIQUE NOT NULL
);

(1 row affected)

Total execution time: 00:00:00.009

In [25]:
CREATE TABLE Management.Bus (
	BusID INT PRIMARY KEY IDENTITY(1, 1),
	PlateNumber VARCHAR(8) UNIQUE NOT NULL CHECK (DATALENGTH(PlateNumber) >= 7),
	Capacity INT NOT NULL CHECK (Capacity > 0),
	StatusID INT REFERENCES Management.BusStatus(StatusID) NOT NULL
);

(1 row affected)

Total execution time: 00:00:00.016

In [26]:
CREATE TABLE Management.Schedule (
	ScheduleID INT PRIMARY KEY IDENTITY(1, 1),
	RouteID INT REFERENCES Management.Route(RouteID) NOT NULL,
	TimeSlotID INT REFERENCES Management.TimeSlot(TimeSlotID) NOT NULL,
	BusID INT REFERENCES Management.Bus(BusID) NOT NULL,
	AvailableCapacity INT NOT NULL CHECK (AvailableCapacity >= 0)
);

(1 row affected)

Total execution time: 00:00:03.036

## Choong Man Shun TP051283

In [27]:
CREATE TABLE BusUser.Student (
	StudentID INT PRIMARY KEY IDENTITY(1, 1),
	Name VARCHAR(200) NOT NULL,
	UserName AS CAST(CONCAT('Student', StudentID) 
		AS VARCHAR(200)) PERSISTED UNIQUE NOT NULL,
	Password VARCHAR(200) NOT NULL,
	EncryptedPasswordByKey VARBINARY(1000),
	EncryptedPasswordByCert VARBINARY(1000)
);

(1 row affected)

Total execution time: 00:00:00.015

In [28]:
CREATE TABLE BusUser.Reservation (
	ReservationID INT PRIMARY KEY IDENTITY(1, 1),
	StudentID INT REFERENCES BusUser.Student(StudentID) NOT NULL,
	ScheduleID INT REFERENCES Management.Schedule(ScheduleID) NOT NULL,
	ConfirmationNumber AS CAST(ReservationID + 100 AS INT) 
		PERSISTED UNIQUE NOT NULL,
	Cancelled BIT NOT NULL DEFAULT 0,
	Datetime DATETIME NOT NULL DEFAULT GETDATE(),
	CONSTRAINT Reservation_AK1 UNIQUE (StudentID, ScheduleID)
);

(1 row affected)

Total execution time: 00:00:00.074

# Implement Column Level Encryption

## Choong Man Shun TP051283

In [29]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Passw0rd';  

Commands completed successfully.

Total execution time: 00:00:00.084

In [30]:
CREATE CERTIFICATE StudentPasswordCertificate 
WITH SUBJECT = 'StudentPassword';

Commands completed successfully.

Total execution time: 00:00:00.286

## Wong Poh Yee TP051079

In [31]:
-- CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Passw0rd';

Commands completed successfully.

Total execution time: 00:00:00.001

In [32]:
CREATE ASYMMETRIC KEY AsymmetricKey 
WITH ALGORITHM = RSA_2048;

Commands completed successfully.

Total execution time: 00:00:00.229

In [33]:
CREATE SYMMETRIC KEY Symmetrickey 
WITH ALGORITHM = AES_256
ENCRYPTION BY ASYMMETRIC KEY AsymmetricKey;

Commands completed successfully.

Total execution time: 00:00:00.002

In [34]:
OPEN SYMMETRIC KEY Symmetrickey
DECRYPTION BY ASYMMETRIC KEY AsymmetricKey;

Commands completed successfully.

Total execution time: 00:00:00.004

# Create Views

## Wong Poh Yee TP051079

In [35]:
CREATE OR ALTER VIEW Management.[Weekly Reporting]
AS
SELECT ScheduleTable.*, Bookings, Cancellations
FROM (
    SELECT
        Route.Name AS Route, 
        YEAR(TimeSlot.DepartureDatetime) AS Year,
        CONCAT('Week ', DATEPART(ISO_WEEK, TimeSlot.DepartureDatetime)) AS Week,
        COUNT(*) AS Schedules,
        COUNT(CASE WHEN TimeSlot.ArrivalDatetime < GETDATE() THEN 1 END) AS [Completed Trips]
    FROM Management.Schedule
    JOIN Management.Route ON Schedule.RouteID = Route.RouteID
    JOIN Management.TimeSlot ON Schedule.TimeSlotID = TimeSlot.TimeSlotID
    GROUP BY 
        Route.Name, 
        YEAR(TimeSlot.DepartureDatetime),
        DATEPART(ISO_WEEK, TimeSlot.DepartureDatetime)
) AS ScheduleTable 
JOIN (
    SELECT 
        Route.Name AS Route, 
        YEAR(TimeSlot.DepartureDatetime) AS Year,
        CONCAT('Week ', DATEPART(ISO_WEEK, TimeSlot.DepartureDatetime)) AS Week,
        COUNT(CASE WHEN Reservation.Cancelled = 0 THEN 1 END) AS Bookings,
        COUNT(CASE WHEN Reservation.Cancelled = 1 THEN 1 END) AS Cancellations
    FROM Management.Schedule
    JOIN Management.Route ON Schedule.RouteID = Route.RouteID
    JOIN Management.TimeSlot ON Schedule.TimeSlotID = TimeSlot.TimeSlotID
    LEFT JOIN BusUser.Reservation ON Schedule.ScheduleID = Reservation.ScheduleID
    GROUP BY 
        Route.Name, 
        YEAR(TimeSlot.DepartureDatetime),
        DATEPART(ISO_WEEK, TimeSlot.DepartureDatetime)
) AS BookingTable
    ON ScheduleTable.Route = BookingTable.Route 
    AND ScheduleTable.Year = BookingTable.Year
    AND ScheduleTable.Week = BookingTable.Week
ORDER BY Route, Year, Week OFFSET 0 ROWS;

(1 row affected)

Total execution time: 00:00:00.131

In [36]:
CREATE OR ALTER VIEW Management.[Monthly Reporting]
AS
SELECT ScheduleTable.*, Bookings, Cancellations
FROM (
    SELECT
        Route.Name AS Route, 
        FORMAT(TimeSlot.DepartureDatetime, 'yyyy-MM') AS Month,
        COUNT(*) AS Schedules,
        COUNT(CASE WHEN TimeSlot.ArrivalDatetime < GETDATE() THEN 1 END) AS [Completed Trips]
    FROM Management.Schedule
    JOIN Management.Route ON Schedule.RouteID = Route.RouteID
    JOIN Management.TimeSlot ON Schedule.TimeSlotID = TimeSlot.TimeSlotID
    GROUP BY 
        Route.Name, 
        FORMAT(TimeSlot.DepartureDatetime, 'yyyy-MM')
) AS ScheduleTable 
JOIN (
    SELECT 
        Route.Name AS Route, 
        FORMAT(TimeSlot.DepartureDatetime, 'yyyy-MM') AS Month,
        COUNT(CASE WHEN Reservation.Cancelled = 0 THEN 1 END) AS Bookings,
        COUNT(CASE WHEN Reservation.Cancelled = 1 THEN 1 END) AS Cancellations
    FROM Management.Schedule
    JOIN Management.Route ON Schedule.RouteID = Route.RouteID
    JOIN Management.TimeSlot ON Schedule.TimeSlotID = TimeSlot.TimeSlotID
    LEFT JOIN BusUser.Reservation ON Schedule.ScheduleID = Reservation.ScheduleID
    GROUP BY 
        Route.Name, 
        FORMAT(TimeSlot.DepartureDatetime, 'yyyy-MM')
) AS BookingTable
    ON ScheduleTable.Route = BookingTable.Route 
    AND ScheduleTable.Month = BookingTable.Month
ORDER BY Route, Month OFFSET 0 ROWS;

(1 row affected)

Total execution time: 00:00:00.065

## Choong Man Shun TP051283

In [37]:
CREATE OR ALTER VIEW BusUser.[Student Profile]
WITH SCHEMABINDING
AS
SELECT 
    StudentID AS [Student ID], 
    Name, 
    UserName,
    EncryptedPasswordByCert AS [Encrypted Password]
FROM BusUser.Student;

(1 row affected)

Total execution time: 00:00:00.161

In [38]:
CREATE OR ALTER VIEW BusUser.[Bus Availability Checking]
WITH SCHEMABINDING
AS
SELECT 
	ScheduleID AS [Schedule ID],
	Route.Name AS Route,
	CAST(DepartureDatetime AS DATE) AS Date,
	CAST(DepartureDatetime AS TIME) AS [Departure Time],
	CAST(ArrivalDatetime AS TIME) AS [Arrival Time],
	PlateNumber AS [Plate Number],
	CONCAT(AvailableCapacity, '/', Capacity) AS Capacity
FROM Management.Schedule
LEFT JOIN Management.Route
	ON Schedule.RouteID = Route.RouteID
LEFT JOIN Management.TimeSlot
	ON Schedule.TimeSlotID = TimeSlot.TimeSlotID
LEFT JOIN Management.Bus
	ON Schedule.BusID = Bus.BusID
WHERE DepartureDatetime BETWEEN GETDATE() 
AND DATEADD(day, 14, GETDATE())
ORDER BY DepartureDatetime, Route.RouteID OFFSET 0 ROWS;

(1 row affected)

Total execution time: 00:00:00.060

In [39]:
CREATE OR ALTER VIEW BusUser.[Student Booking]
WITH SCHEMABINDING
AS
SELECT 
	Reservation.StudentID AS [Student ID], 
	Student.Name, 
    UserName,
	ReservationID AS [Reservation ID], 
	ConfirmationNumber AS [Confirmation Number], 
	Cancelled, 
	Datetime AS [Reservation Datetime],
	Reservation.ScheduleID AS [Schedule ID],
	Route.Name AS Route,
	CAST(DepartureDatetime AS DATE) AS Date,
	CAST(DepartureDatetime AS TIME) AS [Departure Time],
	CAST(ArrivalDatetime AS TIME) AS [Arrival Time],
	PlateNumber AS [Plate Number],
	BusStatus.Name AS [Bus Status]
FROM BusUser.Student
JOIN BusUser.Reservation
	ON Student.StudentID = Reservation.StudentID
LEFT JOIN Management.Schedule
	ON Reservation.ScheduleID = Schedule.ScheduleID
LEFT JOIN Management.Route
	ON Schedule.RouteID = Route.RouteID
LEFT JOIN Management.TimeSlot
	ON Schedule.TimeSlotID = TimeSlot.TimeSlotID
LEFT JOIN Management.Bus ON Schedule.BusID = Bus.BusID
LEFT JOIN Management.BusStatus 
	ON Bus.StatusID = BusStatus.StatusID;

(1 row affected)

Total execution time: 00:00:00.190

In [40]:
CREATE OR ALTER VIEW BusUser.[Student Booking Cancelling]
WITH SCHEMABINDING
AS
SELECT 
	Student.StudentID AS [Student ID], 
	Student.Name, 
    UserName,
	ReservationID AS [Reservation ID], 
	ConfirmationNumber AS [Confirmation Number], 
	Cancelled, 
	Datetime AS [Reservation Datetime],
	Route.Name AS Route,
	DepartureDatetime AS [Departure Datetime],
	ArrivalDatetime AS [Arrival Datetime],
	PlateNumber AS [Plate Number]
FROM BusUser.Student
JOIN BusUser.Reservation
	ON Student.StudentID = Reservation.StudentID
LEFT JOIN Management.Schedule
	ON Reservation.ScheduleID = Schedule.ScheduleID
LEFT JOIN Management.Route
	ON Schedule.RouteID = Route.RouteID
LEFT JOIN Management.TimeSlot
	ON Schedule.TimeSlotID = TimeSlot.TimeSlotID
LEFT JOIN Management.Bus ON Schedule.BusID = Bus.BusID
WHERE DepartureDatetime > GETDATE();

(1 row affected)

Total execution time: 00:00:00.081

# Create DML Triggers

## Wong Poh Yee TP051079

In [41]:
CREATE TABLE Management.AuditLog_Schedule (
	AuditLogID INT IDENTITY(1, 1) NOT NULL,
	LogDate DATETIME DEFAULT GETDATE(),
	UserName SYSNAME DEFAULT USER_NAME(),
	ScheduleID INT,
	RouteID INT,
	TimeSlotID INT,
	BusID INT,
	AvailableCapacity INT,
	UserAction VARCHAR(20)
)

(1 row affected)

Total execution time: 00:00:00.160

In [42]:
CREATE or ALTER TRIGGER Management.AuditScheduleChange
ON Management.Schedule
AFTER INSERT, UPDATE, DELETE
AS 
BEGIN
    INSERT INTO Management.AuditLog_Schedule 
    (ScheduleID, RouteID, TimeSlotID, BusID, AvailableCapacity, UserAction)
    SELECT ScheduleID, RouteID, TimeSlotID, BusID, AvailableCapacity, 'INSERT'
    FROM inserted

    INSERT INTO Management.AuditLog_Schedule
    (ScheduleID, RouteID, TimeSlotID, BusID, AvailableCapacity, UserAction)
    SELECT ScheduleID, RouteID, TimeSlotID, BusID, AvailableCapacity, 'DELETE'
    FROM deleted 
END;

Commands completed successfully.

Total execution time: 00:00:00.003

In [43]:
CREATE TABLE Management.AuditLog_Route (
	AuditLogID INT IDENTITY(1, 1) NOT NULL,
	LogDate DATETIME DEFAULT GETDATE(),
	UserName SYSNAME DEFAULT USER_NAME(),
	RouteID INT,
	DepartureStationID INT,
	ArrivalStationID INT,
	Name VARCHAR(200),
	UserAction VARCHAR(20)
)

(1 row affected)

Total execution time: 00:00:00.565

In [44]:
CREATE or ALTER TRIGGER Management.AuditRouteChange
ON Management.Route
AFTER INSERT, UPDATE, DELETE
AS 
BEGIN
    INSERT INTO Management.AuditLog_Route 
    (RouteID, DepartureStationID, ArrivalStationID, Name, UserAction)
    SELECT RouteID, DepartureStationID, ArrivalStationID, Name, 'INSERT'
    FROM inserted

    INSERT INTO Management.AuditLog_Route
    (RouteID, DepartureStationID, ArrivalStationID, Name, UserAction)
    SELECT RouteID, DepartureStationID, ArrivalStationID, Name, 'DELETE'
    FROM deleted 
END;

Commands completed successfully.

Total execution time: 00:00:00.672

In [45]:
CREATE TABLE Management.AuditLog_TimeSlot (
	AuditLogID INT IDENTITY(1, 1) NOT NULL,
	LogDate DATETIME DEFAULT GETDATE(),
	UserName SYSNAME DEFAULT USER_NAME(),
	TimeSlotID INT,
	DepartureDatetime DATETIME,
	ArrivalDatetime DATETIME,
	UserAction VARCHAR(20)
)

(1 row affected)

Total execution time: 00:00:00.087

In [46]:
CREATE or ALTER TRIGGER Management.AuditTimeSlotChange
ON Management.TimeSlot
AFTER INSERT, UPDATE, DELETE
AS 
BEGIN
    INSERT INTO Management.AuditLog_TimeSlot 
    (TimeSlotID, DepartureDatetime, ArrivalDatetime, UserAction)
    SELECT TimeSlotID, DepartureDatetime, ArrivalDatetime, 'INSERT'
    FROM inserted

    INSERT INTO Management.AuditLog_TimeSlot
    (TimeSlotID, DepartureDatetime, ArrivalDatetime, UserAction)
    SELECT TimeSlotID, DepartureDatetime, ArrivalDatetime, 'DELETE'
    FROM deleted 
END;

Commands completed successfully.

Total execution time: 00:00:00.003

## Choong Man Shun TP051283

In [47]:
CREATE TABLE BusUser.AuditLog_Student (
	AuditLogID INT IDENTITY(1, 1) NOT NULL,
	LogDate DATETIME DEFAULT GETDATE(),
	AuditUserName SYSNAME DEFAULT USER_NAME(),
	StudentID INT,
	Name VARCHAR(200),
	UserName VARCHAR(200),
	Password VARCHAR(200),
	UserAction VARCHAR(20)
);

(1 row affected)

Total execution time: 00:00:02.534

In [48]:
CREATE or ALTER TRIGGER BusUser.AuditStudentChange
ON BusUser.Student
AFTER INSERT, UPDATE, DELETE
AS 
BEGIN
    INSERT INTO BusUser.AuditLog_Student 
    (StudentID, Name, UserName, Password, UserAction)
    SELECT StudentID, Name, UserName, Password, 'INSERT'
    FROM inserted

    INSERT INTO BusUser.AuditLog_Student
    (StudentID, Name, UserName, Password, UserAction)
    SELECT StudentID, Name, UserName, Password, 'DELETE'
    FROM deleted 
END;

Commands completed successfully.

Total execution time: 00:00:00.002

In [49]:
CREATE OR ALTER TRIGGER BusUser.HandleReservation
ON BusUser.Reservation
INSTEAD OF INSERT
AS
BEGIN
    UPDATE Management.Schedule
    SET AvailableCapacity = AvailableCapacity - (
        SELECT COUNT(*) 
        FROM inserted 
        WHERE ScheduleID = Schedule.ScheduleID
    )
    WHERE ScheduleID IN (SELECT ScheduleID FROM inserted);

    INSERT BusUser.Reservation (StudentID, ScheduleID)
    SELECT StudentID, ScheduleID FROM inserted;
END;

Commands completed successfully.

Total execution time: 00:00:00.005

In [50]:
CREATE OR ALTER TRIGGER BusUser.HandleCancellation
ON BusUser.Reservation
INSTEAD OF UPDATE
AS
BEGIN
    DECLARE @rowCount AS INT;
    SET @rowCount = (SELECT COUNT(*) FROM inserted);

    IF @rowCount = 0
        PRINT '0 reservation updated.'
    ELSE IF @rowCount > 1
        PRINT 'Update for multiple reservations is not allowed.'
    ELSE
    BEGIN
        DECLARE @reservationID AS INT;
        SET @reservationID = (SELECT ReservationID FROM inserted);
        
        DECLARE @cancelled AS BIT;
        SET @cancelled = (SELECT Cancelled FROM inserted);
        
        DECLARE @scheduleID AS INT;
        SET @scheduleID = (SELECT ScheduleID FROM inserted);
        
        IF @cancelled = (SELECT Cancelled FROM deleted)
            PRINT CONCAT(
                'Reservation (ID=', 
                @reservationID, 
                ') is ALREADY ', 
                CASE WHEN @cancelled = 0 THEN 'not ' END, 
                'cancelled.'
            )
        ELSE
        BEGIN
            UPDATE Management.Schedule
            SET AvailableCapacity = (
                SELECT AvailableCapacity 
                FROM Management.Schedule 
                WHERE ScheduleID = @scheduleID
            ) + CASE WHEN @cancelled = 1 THEN 1 ELSE -1 END
            WHERE ScheduleID = @scheduleID;

            UPDATE BusUser.Reservation 
            SET Cancelled = @cancelled
            WHERE ReservationID = @reservationID;
            PRINT CONCAT(
                'Reservation (ID=', 
                @reservationID, 
                ') is NOW ', 
                CASE WHEN @cancelled = 0 THEN 'not ' END, 
                'cancelled.'
            );
        END
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.007

# Insert into Tables

## Wong Poh Yee TP051079

In [51]:
INSERT INTO Management.Station
VALUES
('APU Main Campus', 'APU'),
('Bukit Jalil LRT Station', 'Bukit Jalil'),
('Serdang KTM Station', 'Serdang KTM'),
('Bandar Tasek Selatan Bus Terminal', 'BTS');

(4 rows affected)

Total execution time: 00:00:00.980

In [52]:
DECLARE @StationShortName1 AS VARCHAR(20), 
		@StationShortName2 AS VARCHAR(20), 
		@StationShortName3 AS VARCHAR(20), 
		@StationShortName4 AS VARCHAR(20);

SELECT @StationShortName1 = (SELECT ShortName FROM Management.Station WHERE StationID = 1), 
	   @StationShortName2 = (SELECT ShortName FROM Management.Station WHERE StationID = 2), 
	   @StationShortName3 = (SELECT ShortName FROM Management.Station WHERE StationID = 3), 
	   @StationShortName4 = (SELECT ShortName FROM Management.Station WHERE StationID = 4);

INSERT INTO Management.Route
VALUES
(1, 2, @StationShortName1 + ' - ' + @StationShortName2),
(2, 1, @StationShortName2 + ' - ' + @StationShortName1),
(1, 3, @StationShortName1 + ' - ' + @StationShortName3),
(3, 1, @StationShortName3 + ' - ' + @StationShortName1),
(1, 4, @StationShortName1 + ' - ' + @StationShortName4),
(4, 1, @StationShortName4 + ' - ' + @StationShortName1);

(6 rows affected)

(0 rows affected)

(6 rows affected)

Total execution time: 00:00:00.813

In [53]:
INSERT INTO Management.TimeSlot
VALUES
('2022-06-01 06:00:00', '2022-06-01 07:00:00'),
('2022-06-01 07:00:00', '2022-06-01 08:00:00'),
('2022-06-01 09:00:00', '2022-06-01 10:00:00'),
('2022-06-01 10:00:00', '2022-06-01 11:00:00'),
('2022-06-01 11:00:00', '2022-06-01 12:00:00'),
('2022-06-01 14:00:00', '2022-06-01 15:00:00'),
('2022-06-01 16:00:00', '2022-06-01 17:00:00'),
('2022-06-01 18:00:00', '2022-06-01 19:00:00'),
('2022-06-01 20:00:00', '2022-06-01 21:00:00'),
('2022-06-01 21:00:00', '2022-06-01 22:00:00'),

('2022-06-08 06:00:00', '2022-06-08 07:00:00'),
('2022-06-08 07:00:00', '2022-06-08 08:00:00'),
('2022-06-08 09:00:00', '2022-06-08 10:00:00'),
('2022-06-08 10:00:00', '2022-06-08 11:00:00'),
('2022-06-08 11:00:00', '2022-06-08 12:00:00'),
('2022-06-08 14:00:00', '2022-06-08 15:00:00'),
('2022-06-08 16:00:00', '2022-06-08 17:00:00'),
('2022-06-08 18:00:00', '2022-06-08 19:00:00'),
('2022-06-08 20:00:00', '2022-06-08 21:00:00'),
('2022-06-08 21:00:00', '2022-06-08 22:00:00'),

('2022-07-22 06:00:00', '2022-07-22 07:00:00'),
('2022-07-22 07:00:00', '2022-07-22 08:00:00'),
('2022-07-22 09:00:00', '2022-07-22 10:00:00'),
('2022-07-22 10:00:00', '2022-07-22 11:00:00'),
('2022-07-22 11:00:00', '2022-07-22 12:00:00'),
('2022-07-22 14:00:00', '2022-07-22 15:00:00'),
('2022-07-22 16:00:00', '2022-07-22 17:00:00'),
('2022-07-22 18:00:00', '2022-07-22 19:00:00'),
('2022-07-22 20:00:00', '2022-07-22 21:00:00'),
('2022-07-22 21:00:00', '2022-07-22 22:00:00'),

('2022-09-03 06:00:00', '2022-09-03 07:00:00'),
('2022-09-03 07:00:00', '2022-09-03 08:00:00'),
('2022-09-03 09:00:00', '2022-09-03 10:00:00'),
('2022-09-03 10:00:00', '2022-09-03 11:00:00'),
('2022-09-03 11:00:00', '2022-09-03 12:00:00'),
('2022-09-03 14:00:00', '2022-09-03 15:00:00'),
('2022-09-03 16:00:00', '2022-09-03 17:00:00'),
('2022-09-03 18:00:00', '2022-09-03 19:00:00'),
('2022-09-03 20:00:00', '2022-09-03 21:00:00'),
('2022-09-03 21:00:00', '2022-09-03 22:00:00'); 

(40 rows affected)

(0 rows affected)

(40 rows affected)

Total execution time: 00:00:00.070

## Chew Cheng Yong TP051338

In [54]:
INSERT INTO Management.BusStatus
VALUES
('Operating'),
('Resting'),
('Down');

(3 rows affected)

Total execution time: 00:00:00.117

In [55]:
DECLARE @DownStatusID AS INT;

SELECT @DownStatusID = (SELECT StatusID FROM Management.BusStatus WHERE Name = 'Down');

INSERT INTO Management.Bus
VALUES
('WB1234A', 40, @DownStatusID),
('WB5678A', 40, @DownStatusID),
('WB1011A', 40, @DownStatusID),
('WB1213A', 40, @DownStatusID),
('WB1415A', 40, @DownStatusID),
('WB1234B', 40, @DownStatusID),
('WBA1234A', 40, @DownStatusID),
('WB3412C', 40, @DownStatusID),
('WA1334A', 40, @DownStatusID),
('WA1134B', 40, @DownStatusID),
('WBD3334A', 40, @DownStatusID),
('WB1256D', 40, @DownStatusID);

(12 rows affected)

Total execution time: 00:00:00.635

In [56]:
INSERT INTO Management.Schedule
SELECT RouteID, TimeSlotID, BusID, 40 
FROM (
	SELECT 1 AS RouteID, 1 AS BusID
	UNION
	SELECT 1, 2
	UNION
	SELECT 2, 3
	UNION
	SELECT 2, 4
	UNION
	SELECT 3, 5
	UNION
	SELECT 3, 6
	UNION
	SELECT 4, 7
	UNION
	SELECT 4, 8
	UNION
	SELECT 5, 9
	UNION
	SELECT 5, 10
	UNION
	SELECT 6, 11
	UNION
	SELECT 6, 12
) AS RouteBus
CROSS JOIN Management.TimeSlot;

(480 rows affected)

(0 rows affected)

(480 rows affected)

Total execution time: 00:00:02.286

## Choong Man Shun TP051283

In [57]:
INSERT INTO BusUser.Student
(Name, Password)
VALUES
('Man Shun', 'Passw0rd'),
('Cheng Yong', 'Passw0rd'),
('Poh Yee', 'Passw0rd'),
('Riya', 'Passw0rd'),
('Ernest', 'Passw0rd'),
('Diamond', 'Passw0rd'),
('Michelle', 'Passw0rd'),
('Crystal', 'Passw0rd'),
('Sukie', 'Passw0rd'),
('Hashim', 'Passw0rd'),

('Sariah', 'Passw0rd'),
('Julie', 'Passw0rd'),
('Nydia', 'Passw0rd'),
('Cristiana', 'Passw0rd'),
('Emma', 'Passw0rd'),
('Nicodema', 'Passw0rd'),
('Jason', 'Passw0rd'),
('Jessica', 'Passw0rd'),
('Gozzo', 'Passw0rd'),
('Sikandar', 'Passw0rd'),

('Esther', 'Passw0rd'),
('Ryousuke', 'Passw0rd'),
('Domitius', 'Passw0rd'),
('Brighid', 'Passw0rd'),
('Epifanio', 'Passw0rd'),
('Abdul', 'Passw0rd'),
('Sigdag', 'Passw0rd'),
('Freyr', 'Passw0rd'),
('Newen', 'Passw0rd'),
('Ralf', 'Passw0rd'),

('Clodagh', 'Passw0rd'),
('Niall', 'Passw0rd'),
('Kyllikki', 'Passw0rd'),
('Nonhelema', 'Passw0rd'),
('Ningal', 'Passw0rd'),
('Filip', 'Passw0rd'),
('Ammar', 'Passw0rd'),
('Kosmas', 'Passw0rd'),
('Jaylene', 'Passw0rd'),
('Edith', 'Passw0rd'),

('Remedios', 'Passw0rd'),
('Izabel', 'Passw0rd'),
('Ally', 'Passw0rd'),
('Odeserundiye', 'Passw0rd'),
('Cai', 'Passw0rd'),
('Jaquan', 'Passw0rd'),
('Sipho', 'Passw0rd'),
('Yash', 'Passw0rd'),
('Goibniu', 'Passw0rd'),
('Lulu', 'Passw0rd');

(50 rows affected)

(0 rows affected)

(50 rows affected)

Total execution time: 00:00:00.034

In [58]:
UPDATE BusUser.Student
SET EncryptedPasswordByCert = ENCRYPTBYCERT(
    CERT_ID('StudentPasswordCertificate'), Password
);

(50 rows affected)

(50 rows affected)

(50 rows affected)

Total execution time: 00:00:00.029

In [59]:
UPDATE BusUser.Student
SET EncryptedPasswordByKey = ENCRYPTBYKEY(
    KEY_GUID('Symmetrickey'), 
    Password, 
    1, 
    HASHBYTES('SHA2_256', CONVERT(varbinary, StudentID))
);

(50 rows affected)

(50 rows affected)

(50 rows affected)

Total execution time: 00:00:00.010

In [60]:
INSERT INTO BusUser.Reservation
(StudentID, ScheduleID)
VALUES 
(1, 1),
(2, 1),
(3, 1),
(4, 1),
(5, 1),
(6, 1),
(7, 1),
(8, 1),
(9, 1),
(10, 1),

(11, 1),
(12, 1),
(13, 1),
(14, 1),
(15, 1),
(16, 1),
(17, 1),
(18, 1),
(19, 1),
(20, 1),

(21, 1),
(22, 1),
(23, 1),
(24, 1),
(25, 1),
(26, 1),
(27, 1),
(28, 1),
(29, 1),
(30, 1),

(31, 1),
(32, 1),
(33, 1),
(34, 1),
(35, 1),
(36, 1),
(37, 1),
(38, 1),
(39, 1),
(40, 1),

(1, 2),
(2, 2),
(3, 2),
(4, 2),
(5, 2),
(6, 2),
(1, 3),
(2, 3),
(1, 4),
(3, 5);

(5 rows affected)

(5 rows affected)

(5 rows affected)

(50 rows affected)

(50 rows affected)

Total execution time: 00:00:16.541

In [61]:
UPDATE BusUser.Reservation
SET Cancelled = 1
WHERE StudentID = 6 AND ScheduleID = 2;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Reservation (ID=46) is NOW cancelled.

(1 row affected)

Total execution time: 00:00:00.014

# Create Roles

## Wong Poh Yee TP051079

In [62]:
CREATE ROLE Management;

Commands completed successfully.

Total execution time: 00:00:00.004

## Chew Cheng Yong TP051338

In [63]:
CREATE ROLE DatabaseAdmins;

Commands completed successfully.

Total execution time: 00:00:00.001

In [64]:
CREATE ROLE Schedulers;

Commands completed successfully.

Total execution time: 00:00:00.002

## Choong Man Shun TP051283

In [65]:
CREATE ROLE Students;

Commands completed successfully.

Total execution time: 00:00:00.001

# Creates Logins

## Wong Poh Yee TP051079

In [66]:
CREATE LOGIN AnalystDeptHead
WITH PASSWORD = 'Passw0rd',
DEFAULT_DATABASE = [APU Bus Services],
CHECK_POLICY = ON,
CHECK_EXPIRATION = OFF;

CREATE LOGIN ServicesDeptHead
WITH PASSWORD = 'Passw0rd',
DEFAULT_DATABASE = [APU Bus Services],
CHECK_POLICY = ON,
CHECK_EXPIRATION = OFF;

Commands completed successfully.

Total execution time: 00:00:00.037

## Chew Cheng Yong TP051338

In [67]:
CREATE LOGIN DatabaseAdmin1
WITH PASSWORD = '$tr0ngP@$$w0rd',
DEFAULT_DATABASE = [APU Bus Services],
CHECK_POLICY = ON,
CHECK_EXPIRATION = OFF;

Commands completed successfully.

Total execution time: 00:00:00.002

In [68]:
CREATE LOGIN Scheduler1
WITH PASSWORD = '$tr0ngP@$$w0rd',
DEFAULT_DATABASE = [APU Bus Services],
CHECK_POLICY = ON,
CHECK_EXPIRATION = OFF;

CREATE LOGIN Scheduler2
WITH PASSWORD = '$tr0ngP@$$w0rd',
DEFAULT_DATABASE = [APU Bus Services],
CHECK_POLICY = ON,
CHECK_EXPIRATION = OFF;

Commands completed successfully.

Total execution time: 00:00:00.003

## Choong Man Shun TP051283

In [69]:
CREATE LOGIN Student1
WITH PASSWORD = 'Passw0rd',
DEFAULT_DATABASE = [APU Bus Services],
CHECK_POLICY = ON,
CHECK_EXPIRATION = OFF;

CREATE LOGIN Student2
WITH PASSWORD = 'Passw0rd',
DEFAULT_DATABASE = [APU Bus Services],
CHECK_POLICY = ON,
CHECK_EXPIRATION = OFF;

CREATE LOGIN Student3
WITH PASSWORD = 'Passw0rd',
DEFAULT_DATABASE = [APU Bus Services],
CHECK_POLICY = ON,
CHECK_EXPIRATION = OFF;

Commands completed successfully.

Total execution time: 00:00:00.035

# Create Users

## Wong Poh Yee TP051079

In [70]:
-- Create management users with login 
CREATE USER AnalystDeptHead FOR LOGIN AnalystDeptHead;
CREATE USER ServicesDeptHead FOR LOGIN ServicesDeptHead;

Commands completed successfully.

Total execution time: 00:00:00.050

In [71]:
-- Create management users without login
CREATE USER BusDeptHead WITHOUT LOGIN;

Commands completed successfully.

Total execution time: 00:00:00.001

## Chew Cheng Yong TP051338

In [72]:
-- Create database admin user with login
CREATE USER DatabaseAdmin1 FOR LOGIN DatabaseAdmin1;

Commands completed successfully.

Total execution time: 00:00:00.001

In [73]:
-- Create database admin user without login
CREATE USER DatabaseAdmin2 WITHOUT LOGIN;
CREATE USER DatabaseAdmin3 WITHOUT LOGIN;

Commands completed successfully.

Total execution time: 00:00:00.008

In [74]:
-- Create scheduler user with login
CREATE USER Scheduler1 FOR LOGIN Scheduler1;
CREATE USER Scheduler2 FOR LOGIN Scheduler2;

Commands completed successfully.

Total execution time: 00:00:00.002

In [75]:
-- Create scheduler user without login
CREATE USER Scheduler3 WITHOUT LOGIN;

Commands completed successfully.

Total execution time: 00:00:00.001

## Choong Man Shun TP051283

In [76]:
-- Create student users with login
CREATE USER Student1 FOR LOGIN Student1;
CREATE USER Student2 FOR LOGIN Student2;
CREATE USER Student3 FOR LOGIN Student3;

Commands completed successfully.

Total execution time: 00:00:00.002

In [77]:
-- Create student users without login
CREATE USER Student4 WITHOUT LOGIN;
CREATE USER Student5 WITHOUT LOGIN;
CREATE USER Student6 WITHOUT LOGIN;
CREATE USER Student7 WITHOUT LOGIN;
CREATE USER Student8 WITHOUT LOGIN;
CREATE USER Student9 WITHOUT LOGIN;
CREATE USER Student10 WITHOUT LOGIN;
CREATE USER Student11 WITHOUT LOGIN;
CREATE USER Student12 WITHOUT LOGIN;
CREATE USER Student13 WITHOUT LOGIN;
CREATE USER Student14 WITHOUT LOGIN;
CREATE USER Student15 WITHOUT LOGIN;
CREATE USER Student16 WITHOUT LOGIN;
CREATE USER Student17 WITHOUT LOGIN;
CREATE USER Student18 WITHOUT LOGIN;
CREATE USER Student19 WITHOUT LOGIN;
CREATE USER Student20 WITHOUT LOGIN;
CREATE USER Student21 WITHOUT LOGIN;
CREATE USER Student22 WITHOUT LOGIN;
CREATE USER Student23 WITHOUT LOGIN;
CREATE USER Student24 WITHOUT LOGIN;
CREATE USER Student25 WITHOUT LOGIN;
CREATE USER Student26 WITHOUT LOGIN;
CREATE USER Student27 WITHOUT LOGIN;
CREATE USER Student28 WITHOUT LOGIN;
CREATE USER Student29 WITHOUT LOGIN;
CREATE USER Student30 WITHOUT LOGIN;
CREATE USER Student31 WITHOUT LOGIN;
CREATE USER Student32 WITHOUT LOGIN;
CREATE USER Student33 WITHOUT LOGIN;
CREATE USER Student34 WITHOUT LOGIN;
CREATE USER Student35 WITHOUT LOGIN;
CREATE USER Student36 WITHOUT LOGIN;
CREATE USER Student37 WITHOUT LOGIN;
CREATE USER Student38 WITHOUT LOGIN;
CREATE USER Student39 WITHOUT LOGIN;
CREATE USER Student40 WITHOUT LOGIN;
CREATE USER Student41 WITHOUT LOGIN;
CREATE USER Student42 WITHOUT LOGIN;
CREATE USER Student43 WITHOUT LOGIN;
CREATE USER Student44 WITHOUT LOGIN;
CREATE USER Student45 WITHOUT LOGIN;
CREATE USER Student46 WITHOUT LOGIN;
CREATE USER Student47 WITHOUT LOGIN;
CREATE USER Student48 WITHOUT LOGIN;
CREATE USER Student49 WITHOUT LOGIN;
CREATE USER Student50 WITHOUT LOGIN;

Commands completed successfully.

Total execution time: 00:00:00.020

# Add Users to Role

## Wong Poh Yee TP051079

In [78]:
ALTER ROLE Management ADD MEMBER BusDeptHead;
ALTER ROLE Management ADD MEMBER AnalystDeptHead;
ALTER ROLE Management ADD MEMBER ServicesDeptHead;

Commands completed successfully.

Total execution time: 00:00:00.004

## Chew Cheng Yong TP051338

In [79]:
ALTER ROLE DatabaseAdmins ADD MEMBER DatabaseAdmin1;
ALTER ROLE DatabaseAdmins ADD MEMBER DatabaseAdmin2;
ALTER ROLE DatabaseAdmins ADD MEMBER DatabaseAdmin3;

Commands completed successfully.

Total execution time: 00:00:00.002

In [80]:
ALTER ROLE Schedulers ADD MEMBER Scheduler1;
ALTER ROLE Schedulers ADD MEMBER Scheduler2;
ALTER ROLE Schedulers ADD MEMBER Scheduler3;

Commands completed successfully.

Total execution time: 00:00:00.002

## Choong Man Shun TP051283

In [81]:
ALTER ROLE Students ADD MEMBER Student1;
ALTER ROLE Students ADD MEMBER Student2;
ALTER ROLE Students ADD MEMBER Student3;
ALTER ROLE Students ADD MEMBER Student4;
ALTER ROLE Students ADD MEMBER Student5;
ALTER ROLE Students ADD MEMBER Student6;
ALTER ROLE Students ADD MEMBER Student7;
ALTER ROLE Students ADD MEMBER Student8;
ALTER ROLE Students ADD MEMBER Student9;
ALTER ROLE Students ADD MEMBER Student10;
ALTER ROLE Students ADD MEMBER Student11;
ALTER ROLE Students ADD MEMBER Student12;
ALTER ROLE Students ADD MEMBER Student13;
ALTER ROLE Students ADD MEMBER Student14;
ALTER ROLE Students ADD MEMBER Student15;
ALTER ROLE Students ADD MEMBER Student16;
ALTER ROLE Students ADD MEMBER Student17;
ALTER ROLE Students ADD MEMBER Student18;
ALTER ROLE Students ADD MEMBER Student19;
ALTER ROLE Students ADD MEMBER Student20;
ALTER ROLE Students ADD MEMBER Student21;
ALTER ROLE Students ADD MEMBER Student22;
ALTER ROLE Students ADD MEMBER Student23;
ALTER ROLE Students ADD MEMBER Student24;
ALTER ROLE Students ADD MEMBER Student25;
ALTER ROLE Students ADD MEMBER Student26;
ALTER ROLE Students ADD MEMBER Student27;
ALTER ROLE Students ADD MEMBER Student28;
ALTER ROLE Students ADD MEMBER Student29;
ALTER ROLE Students ADD MEMBER Student30;
ALTER ROLE Students ADD MEMBER Student31;
ALTER ROLE Students ADD MEMBER Student32;
ALTER ROLE Students ADD MEMBER Student33;
ALTER ROLE Students ADD MEMBER Student34;
ALTER ROLE Students ADD MEMBER Student35;
ALTER ROLE Students ADD MEMBER Student36;
ALTER ROLE Students ADD MEMBER Student37;
ALTER ROLE Students ADD MEMBER Student38;
ALTER ROLE Students ADD MEMBER Student39;
ALTER ROLE Students ADD MEMBER Student40;
ALTER ROLE Students ADD MEMBER Student41;
ALTER ROLE Students ADD MEMBER Student42;
ALTER ROLE Students ADD MEMBER Student43;
ALTER ROLE Students ADD MEMBER Student44;
ALTER ROLE Students ADD MEMBER Student45;
ALTER ROLE Students ADD MEMBER Student46;
ALTER ROLE Students ADD MEMBER Student47;
ALTER ROLE Students ADD MEMBER Student48;
ALTER ROLE Students ADD MEMBER Student49;
ALTER ROLE Students ADD MEMBER Student50;

Commands completed successfully.

Total execution time: 00:00:00.086

# Implement Object Level Security

## Wong Poh Yee TP051079

In [82]:
GRANT SELECT ON Management.[Weekly Reporting] TO Management;

Commands completed successfully.

Total execution time: 00:00:00.024

In [83]:
GRANT SELECT ON Management.[Monthly Reporting] TO Management;

Commands completed successfully.

Total execution time: 00:00:00.002

## Chew Cheng Yong TP051338

In [84]:
GRANT CONTROL
ON SCHEMA::[Management]
TO DatabaseAdmins
WITH GRANT OPTION;

GRANT CONTROL
ON SCHEMA::[BusUser]
TO DatabaseAdmins
WITH GRANT OPTION;

--once grant create permission, then it can perform delete/drop
GRANT CREATE VIEW TO [DatabaseAdmins]
GRANT CREATE TABLE TO [DatabaseAdmins]

Commands completed successfully.

Total execution time: 00:00:00.002

In [85]:
GRANT SELECT, INSERT, UPDATE, DELETE
ON Management.Route
TO Schedulers;

GRANT SELECT, INSERT, UPDATE, DELETE
ON Management.Schedule
TO Schedulers;

GRANT SELECT, INSERT, UPDATE, DELETE
ON Management.TimeSlot
TO Schedulers;

GRANT SELECT, INSERT, UPDATE
ON Management.Station
TO Schedulers;

GRANT SELECT, INSERT, UPDATE
ON Management.Bus
TO Schedulers;

GRANT SELECT
ON Management.BusStatus
TO Schedulers;

Commands completed successfully.

Total execution time: 00:00:00.048

## Choong Man Shun TP051283

In [86]:
GRANT SELECT, UPDATE ON BusUser.[Student Profile] TO Students;

Commands completed successfully.

Total execution time: 00:00:00.001

In [87]:
GRANT SELECT ON BusUser.[Bus Availability Checking] TO Students;

Commands completed successfully.

Total execution time: 00:00:00.001

In [88]:
GRANT SELECT, INSERT ON BusUser.[Student Booking] TO Students;

Commands completed successfully.

Total execution time: 00:00:00.001

In [89]:
GRANT SELECT ON BusUser.[Student Booking Cancelling] TO Students;

Commands completed successfully.

Total execution time: 00:00:00.003

In [90]:
GRANT UPDATE ON BusUser.[Student Booking Cancelling](Cancelled) TO Students;

Commands completed successfully.

Total execution time: 00:00:00.001

# Implement Row Level Security (RLS)

## Choong Man Shun (TP051283)

In [91]:
CREATE SCHEMA Security;

Commands completed successfully.

Total execution time: 00:00:00.001

In [92]:
CREATE FUNCTION Security.tvf_securitypredicate
(@UserName AS nvarchar(100))
    RETURNS TABLE
WITH SCHEMABINDING
AS
    RETURN SELECT 1 AS tvf_securitypredicate_result
WHERE @UserName = USER_NAME() 
OR USER_NAME() = 'dbo'
OR IS_MEMBER('DatabaseAdmins') = 1;

Commands completed successfully.

Total execution time: 00:00:00.029

In [93]:
CREATE SECURITY POLICY Security.StudentProfileFilter
ADD FILTER PREDICATE 
    Security.tvf_securitypredicate(UserName)
ON BusUser.[Student Profile];

Commands completed successfully.

Total execution time: 00:00:00.006

In [94]:
CREATE SECURITY POLICY Security.StudentBookingFilter
ADD FILTER PREDICATE 
    Security.tvf_securitypredicate(UserName)
ON BusUser.[Student Booking];

Commands completed successfully.

Total execution time: 00:00:00.003

In [95]:
CREATE SECURITY POLICY Security.StudentBookingCancellingFilter
ADD FILTER PREDICATE 
    Security.tvf_securitypredicate(UserName)
ON BusUser.[Student Booking Cancelling];

Commands completed successfully.

Total execution time: 00:00:00.005

# Create Logon Triggers

## Wong Poh Yee TP051079

In [96]:
CREATE OR ALTER TRIGGER LimitSessions 
ON ALL SERVER
FOR LOGON 
AS
BEGIN
    IF ORIGINAL_LOGIN() LIKE 'AnalystDeptHead' OR ORIGINAL_LOGIN() LIKE 'ServicesDeptHead' 
    AND (
        SELECT COUNT(*) FROM sys.dm_exec_sessions 
        WHERE is_user_process = 1 
        AND original_login_name = ORIGINAL_LOGIN()
    ) > 5
    BEGIN
        PRINT 'Maximum connection allowed per user is 5 only';
        ROLLBACK;
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.202

In [97]:
CREATE OR ALTER TRIGGER LimitManagementLoginHours 
ON ALL SERVER
FOR LOGON 
AS
BEGIN
    IF ORIGINAL_LOGIN() LIKE 'AnalystDeptHead' OR ORIGINAL_LOGIN() LIKE 'ServicesDeptHead' 
    AND DATEPART(HOUR, GETDATE()) BETWEEN 0 AND 5
    BEGIN
        PRINT 'You are only allowed to log in after 6am and before 12am.';
        ROLLBACK;
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.002

## Chew Cheng Yong TP051338

In [98]:
CREATE OR ALTER TRIGGER LimitSchedulerSessions 
ON ALL SERVER
FOR LOGON 
AS
BEGIN
    IF ORIGINAL_LOGIN() LIKE 'Scheduler%'
    AND (
        SELECT COUNT(*) FROM sys.dm_exec_sessions 
        WHERE is_user_process = 1 
        AND original_login_name = ORIGINAL_LOGIN()
    ) > 5
    BEGIN
        PRINT 'Maximum connection allowed per scheduler user is 5 only.';
        ROLLBACK;
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.002

In [99]:
CREATE OR ALTER TRIGGER LimitDatabaseAdminsSessions 
ON ALL SERVER
FOR LOGON 
AS
BEGIN
    IF ORIGINAL_LOGIN() LIKE 'DatabaseAdmin%'
    AND (
        SELECT COUNT(*) FROM sys.dm_exec_sessions 
        WHERE is_user_process = 1 
        AND original_login_name = ORIGINAL_LOGIN()
    ) > 5
    BEGIN
        PRINT 'Maximum connection allowed per database administrator is 5 only.';
        ROLLBACK;
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.002

In [100]:
--add ur host_name() inside the list, if not ltr cannot log in
CREATE OR ALTER TRIGGER MyHostsOnly
ON ALL SERVER
FOR LOGON
AS
BEGIN
    -- White list of allowed hostnames are defined here.
    IF HOST_NAME() NOT IN ('ProdBox', 'QaBox', 'DevBox', 'UserBox', 'LAPTOP-1QS5DC0S', 'MANSHUN', 'Evangelines-M1-Air')
    BEGIN
        RAISERROR('You are not allowed to login from this hostname.', 16, 1);
        ROLLBACK;
    END 
END

Commands completed successfully.

Total execution time: 00:00:00.001

In [101]:
--add ur suser_name() inside the list, if not ltr cannot log in
CREATE OR ALTER TRIGGER LimitConnectionAfterOfficeHours
ON ALL SERVER FOR LOGON 
AS
BEGIN
    IF ORIGINAL_LOGIN() IN ('Scheduler1', 'Scheduler2', 'Scheduler3') 
    AND SUSER_NAME() NOT IN ('sa','LAPTOP-1QS5DC0S\chewc') 
    AND (DATEPART(HOUR, GETDATE()) < 8 
    OR DATEPART (HOUR, GETDATE()) > 18)
    BEGIN
        PRINT 'You are not authorized to login after office hours';
        ROLLBACK;
    END
END

Commands completed successfully.

Total execution time: 00:00:00.001

## Choong Man Shun (TP051283)

In [102]:
CREATE TABLE Management.AuditLog_Logon (
	AuditLogID INT IDENTITY(1, 1) NOT NULL,
	LogDate DATETIME DEFAULT GETDATE(),
	UserName SYSNAME DEFAULT ORIGINAL_LOGIN(),
	Spid SMALLINT
);

(1 row affected)

Total execution time: 00:00:00.035

In [103]:
GRANT INSERT ON Management.AuditLog_Logon TO Students;
GRANT INSERT ON Management.AuditLog_Logon TO Management;
GRANT INSERT ON Management.AuditLog_Logon TO DatabaseAdmins 
WITH GRANT OPTION;
GRANT INSERT ON Management.AuditLog_Logon TO Schedulers;

Commands completed successfully.

Total execution time: 00:00:00.003

In [104]:
CREATE OR ALTER TRIGGER AuditLogon
ON ALL SERVER 
FOR LOGON 
AS
    INSERT INTO [APU Bus Services].Management.AuditLog_Logon 
    (Spid)
    VALUES (@@SPID);

Commands completed successfully.

Total execution time: 00:00:00.066

In [105]:
CREATE OR ALTER TRIGGER LimitStudentSessions 
ON ALL SERVER
FOR LOGON 
AS
BEGIN
    IF ORIGINAL_LOGIN() LIKE 'Student%'
    AND (
        SELECT COUNT(*) FROM sys.dm_exec_sessions 
        WHERE is_user_process = 1 
        AND original_login_name = ORIGINAL_LOGIN()
    ) > 3
    BEGIN
        PRINT 'Maximum connection allowed per student user is 3 only.';
        ROLLBACK;
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.002

In [106]:
CREATE OR ALTER TRIGGER LimitStudentLoginHours 
ON ALL SERVER
FOR LOGON 
AS
BEGIN
    IF ORIGINAL_LOGIN() LIKE 'Student%'
    AND DATEPART(HOUR, GETDATE()) NOT BETWEEN 6 AND 22
    BEGIN
        PRINT 'You are only allowed to log in after 6am and before 10pm.';
        ROLLBACK;
    END
END;

Commands completed successfully.

Total execution time: 00:00:00.001

# Extra

In [107]:
SELECT 
	StudentID, 
	Name, 
	EncryptedPasswordByCert, 
	CAST(
		DECRYPTBYCERT(
			CERT_ID('StudentPasswordCertificate'), 
			EncryptedPasswordByCert
		) AS VARCHAR
	) AS DecryptedPasswordByCert,
	EncryptedPasswordByKey,
	CAST(
		DECRYPTBYKEY(
			EncryptedPasswordByKey, 
			1, 
			HASHBYTES('SHA2_256', CONVERT(varbinary, StudentID))
		) AS VARCHAR
	) AS DecryptedPasswordByKey
FROM BusUser.Student;

(50 rows affected)

Total execution time: 00:00:00.216

StudentID,Name,EncryptedPasswordByCert,DecryptedPasswordByCert,EncryptedPasswordByKey,DecryptedPasswordByKey
1,Man Shun,0x4E0D3FF8BEF91ECE5DB8A79DFBAD5D77C3A4632F73963E22AA961F612204A816C90D19F5C19417312F10F550546E7BE8495C310AA6B0524B6232FC46CC04CA12C4082D96B4A42010EF18D55882A00BCE72DE02C64DC1E8E913BE9DD2C50D7B8B586FB01D85DC001569005311E1D74C16A1B28A3C7228A3AEEBC9FEC6664DEC46289F1F877143F95A89A12866F688A992B7F9EDC1B9F21C665512DE0A61DDBAB25AD7CDC22EE522F28E58C75CD1F3A4BF5A3423975DF46F1C7A7CB08856BF04BF6C3EBC5F76DAB83816637F3A9C34E8C49300A7C6841F2ED69BFD6688B2572CB85E9D7116B809373DF5F27F034953D46242605EAF8744B4CB97F6A070A6038389,Passw0rd,0x007BFB3E9C053A4991CB5F9E4658551F0200000099ABAFC5D256779F7DCBF8D8ECCC7CDF13F47632882E4FBF91E0519198471601656ACE4E5FE3932E740B7ABAF553C7AC0BA4F2ACFCA1075AD97D344C264BE4BC,Passw0rd
2,Cheng Yong,0x5519D2291CB7E14B302DED37E06103A3A114DDA99697C69F75F6B33804E6E9B671F45962BBCF23FE40F3DA40CB85088B37319A1CF30998D28C8222D46CA6C228D21F80435672B3B4C2718A4BC1AB2F2839434CDD6627B9EE29527B5BEA07AEF6FBDEA6AE6DEB71F1F7F9ED6882E82154B7DFE52D5DEFCED91046E5882E65D7CC3EBE97E817D808E5CB8B4841A535ABEA70BD58A653F79805C624BB47F0CE474DA7403EF55C7F7C82BD5F6E0B7D497CB2C334459E70498D945A1DEDC4C7280225ACAACF92BC60746A40A6FD8EB3D72ADB0D38D1D4E8F54281552A4A8FBA52BB6B823CB6F41602FA42C2051BEF60C9E51C0F3D57876CA2A3A1602BC2E79F05C7B5,Passw0rd,0x007BFB3E9C053A4991CB5F9E4658551F02000000C0A1496B28930D5B07AC78AB975758F2BDC67546E3777834B7AF0C8FDD641A8A190DD1E4FEF731268F4958CBD199EDBA806C5C06B5A3A5A4824E8E8795525376,Passw0rd
3,Poh Yee,0xC04710FF3B604A56FCE70CC6FF9D8A88933A50C5FF9E974DCD9C1ACDB1A64A6D290FFA21B6F9777F96AE160E7B6138962B93102E28D3C1C8B72C5C888E592F31C8C4DF6B0684619A72474007F6BD2AB2BE7A7EDA91474622F2AA159A39DC000A94A2341669A5947F2F3AFE243F9C13AB55A9170D24B7BD08DE6490B07B31B6384688F492271A4C9BD85E1CA83387D335D46A0700B2E1DCDC7B85ACF4BC929114B49E0AD77A07721F04A16248A662DB9578B6654A67A395C36186CFB99527218F9F9DE2B5B17C9693AFB6888FD168102FCB2EB2BFE69A256B467FF2A1823CAEF2950C5D123DF990C6B5410FEEDD7D8699073579A456DA0B46FC565927F7C02937,Passw0rd,0x007BFB3E9C053A4991CB5F9E4658551F02000000DF34EE3656466C0C3C22A0C7FCD612FEBD1EF463102BBF53FB184FBC5C54AF3BA81196C6A29C9AB0CF00FDFBF0F90EC36A86073808701C3E33D39B14EA160334,Passw0rd
4,Riya,0xF0A9D25EC36FA89C7A352B9D58E27FEA556319D0628100FB28E52FB5FF3AAC6657F2AFFB08F925C0BEF4ECFDAACFB4024E61680AD97722398F37880A2B49770BD671EEFBA186D617765384351CC6E4F974160A960C9A3932E0325D95268E1811724B57F0E6781417F1068F5676869317E1EA83BAE6FFFCF225DA8E0DBD9CF9B4BE0E7CF87E3716AF278A19B127DED0E4C1E6C354E8A099A274AD487E58CBB52F79E5A24DFE0935CD0321D93F253109EFEDDF3F4F03360FFE82B49BCBDDA76B9293637BE51207FB341417CE89153D5370926C39D1FAE3CFE74D9764AF9CBA7B5EEA1811E0693A29EEDD36C8AC0771EAF2AB0A29527CE1148A19C68CD4139C1916,Passw0rd,0x007BFB3E9C053A4991CB5F9E4658551F0200000085DDB8A8EABDDAF63CC33189AC8E6AA12519B414C9D6C245B2AABBFF6292602F0F83B8C93C6356A9CB5530DF2801E0C3E6EA224B5EFEA58EC96FE0584DCA79E7,Passw0rd
5,Ernest,0x36F86CD5D93F2C44A66D31F727D00F115466A118671824976992810726610019E7E19EFC1F1E3435C031F9A9AA2DAC753F8C7C3C5BF67BF59EA7A7D6DFD1637868CF4E4447491847C1D0E1346101FF493B2980638EB591DB91E07616D0889CE61352979EED89453A5F66E58979F906E4978E19773C1FEA0CE3184528F91A3746C49710ADF22C2D9499A274B0F87F28F70E303D72AB971024E3310CDCD986B0FBA7F4979371C7440C278F85EDF1E2109F7845C6FC3CA61606E9AB8C5FDE32B2987948971318338C5D4472B320A531747EB239E94B8322F8AC28C92C0965F745086996174528D4B97DF5397C4FB305B606E7C4C424519618798F30668A8B860FAB,Passw0rd,0x007BFB3E9C053A4991CB5F9E4658551F020000007ABB742E750DCE974D401172C4760DF4AA616BF4B60629273340FA82802CF1C0834F1D4B0C58817D7BC5166D0334481334CD171A2BE6F3FF7C93B180171C9333,Passw0rd
6,Diamond,0x8F97A745B83FB6CDBDC104886515BC1B5E9313F8834C958DF024ADCFD5AB5A87573418C572685AF02735C1CC1BB3CF33B4E7FF4C3B30014C55EE83B71041BE4A317A31B36DBC77589EAF072E6682052D7E4AE5A186E3D8FCF73825E9CCECD28AE9078E9F1249C26EBC2F8E51FA99C83AD77B84C120D8ECD4BDD21786B706C3D2296A5BB6EDE95E75382F7B2E6D5B79E5217A144D4A61EFD23D0B1

In [108]:
SELECT 
	ScheduleID, 
	CASE
		WHEN Cancelled = 1
		THEN 'Yes'
		ELSE 'No'
	END AS Cancelled, 
	COUNT(*) AS Count
FROM BusUser.Reservation
GROUP BY Cancelled, ScheduleID;

(6 rows affected)

Total execution time: 00:00:00.174

ScheduleID,Cancelled,Count
1,No,40
2,No,5
2,Yes,1
3,No,2
4,No,1
5,No,1


In [109]:
SELECT 
    Schedule.*, 
    COUNT(Reservation.ReservationID) AS ReservationCount 
FROM Management.Schedule 
JOIN (
    SELECT * FROM BusUser.Reservation WHERE Cancelled = 0
) AS Reservation
    ON Schedule.ScheduleID = Reservation.ScheduleID 
GROUP BY 
    Schedule.ScheduleID, RouteID, TimeSlotID, BusID, AvailableCapacity;

(5 rows affected)

Total execution time: 00:00:00.053

ScheduleID,RouteID,TimeSlotID,BusID,AvailableCapacity,ReservationCount
1,1,1,1,0,40
2,1,2,1,35,5
3,1,3,1,38,2
4,1,4,1,39,1
5,1,5,1,39,1


In [110]:
-- -- Exceed capacity (not allowed)
-- INSERT INTO BusUser.Reservation
-- (StudentID, ScheduleID)
-- VALUES 
-- (50, 1);

Commands completed successfully.

Total execution time: 00:00:00

## Student

In [111]:
-- Student: View SQL student table (not allowed)
-- EXECUTE AS USER = 'Student1';
-- SELECT * FROM BusUser.Student;
-- REVERT;

Commands completed successfully.

Total execution time: 00:00:00

In [112]:
-- Student: View profile
EXECUTE AS USER = 'Student1';
PRINT USER_NAME();
SELECT * FROM BusUser.[Student Profile];
REVERT;

Student1

(1 row affected)

Total execution time: 00:00:00.043

Student ID,Name,UserName,Encrypted Password
1,Man Shun,Student1,0x4E0D3FF8BEF91ECE5DB8A79DFBAD5D77C3A4632F73963E22AA961F612204A816C90D19F5C19417312F10F550546E7BE8495C310AA6B0524B6232FC46CC04CA12C4082D96B4A42010EF18D55882A00BCE72DE02C64DC1E8E913BE9DD2C50D7B8B586FB01D85DC001569005311E1D74C16A1B28A3C7228A3AEEBC9FEC6664DEC46289F1F877143F95A89A12866F688A992B7F9EDC1B9F21C665512DE0A61DDBAB25AD7CDC22EE522F28E58C75CD1F3A4BF5A3423975DF46F1C7A7CB08856BF04BF6C3EBC5F76DAB83816637F3A9C34E8C49300A7C6841F2ED69BFD6688B2572CB85E9D7116B809373DF5F27F034953D46242605EAF8744B4CB97F6A070A6038389


In [113]:
-- Student: Change profile password
DECLARE @newPassword VARBINARY(2000);
SELECT @newPassword = ENCRYPTBYCERT(CERT_ID('StudentPasswordCertificate'), 'NewPassw0rd');

EXECUTE AS USER = 'Student1';

UPDATE BusUser.[Student Profile] 
SET [Encrypted Password] = @newPassword
WHERE [Student ID] = 1;

REVERT;

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.082

In [114]:
SELECT * FROM BusUser.[Bus Availability Checking];

(120 rows affected)

Total execution time: 00:00:00.028

Schedule ID,Route,Date,Departure Time,Arrival Time,Plate Number,Capacity
21,APU - Bukit Jalil,2022-07-22,06:00:00,07:00:00,WB1234A,40/40
61,APU - Bukit Jalil,2022-07-22,06:00:00,07:00:00,WB5678A,40/40
101,Bukit Jalil - APU,2022-07-22,06:00:00,07:00:00,WB1011A,40/40
141,Bukit Jalil - APU,2022-07-22,06:00:00,07:00:00,WB1213A,40/40
181,APU - Serdang KTM,2022-07-22,06:00:00,07:00:00,WB1415A,40/40
221,APU - Serdang KTM,2022-07-22,06:00:00,07:00:00,WB1234B,40/40
261,Serdang KTM - APU,2022-07-22,06:00:00,07:00:00,WBA1234A,40/40
301,Serdang KTM - APU,2022-07-22,06:00:00,07:00:00,WB3412C,40/40
341,APU - BTS,2022-07-22,06:00:00,07:00:00,WA1334A,40/40
381,APU - BTS,2022-07-22,06:00:00,07:00:00,WA1134B,40/40


In [115]:
-- Student: Place booking
EXECUTE AS USER = 'Student1';

INSERT BusUser.[Student Booking]
([Student ID], [Schedule ID])
VALUES
(1, 101);

REVERT;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.024

In [116]:
-- Student: View booking
EXECUTE AS USER = 'Student1';

SELECT * FROM BusUser.[Student Booking];

REVERT;

(5 rows affected)

Total execution time: 00:00:00.029

Student ID,Name,UserName,Reservation ID,Confirmation Number,Cancelled,Reservation Datetime,Schedule ID,Route,Date,Departure Time,Arrival Time,Plate Number,Bus Status
1,Man Shun,Student1,1,101,0,2022-07-13 02:51:05.683,1,APU - Bukit Jalil,2022-06-01,06:00:00,07:00:00,WB1234A,Down
1,Man Shun,Student1,41,141,0,2022-07-13 02:51:05.683,2,APU - Bukit Jalil,2022-06-01,07:00:00,08:00:00,WB1234A,Down
1,Man Shun,Student1,47,147,0,2022-07-13 02:51:05.683,3,APU - Bukit Jalil,2022-06-01,09:00:00,10:00:00,WB1234A,Down
1,Man Shun,Student1,49,149,0,2022-07-13 02:51:05.683,4,APU - Bukit Jalil,2022-06-01,10:00:00,11:00:00,WB1234A,Down
1,Man Shun,Student1,51,151,0,2022-07-13 02:51:33.170,101,Bukit Jalil - APU,2022-07-22,06:00:00,07:00:00,WB1011A,Down


In [117]:
-- Student: Cancel booking
EXECUTE AS USER = 'Student1';

UPDATE BusUser.[Student Booking Cancelling]
SET Cancelled = 1
WHERE [Reservation ID] = 51;

REVERT;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Reservation (ID=51) is NOW cancelled.

(1 row affected)

Total execution time: 00:00:00.057

## Managment

In [118]:
EXECUTE AS USER = 'AnalystDeptHead';
SELECT * FROM Management.[Monthly Reporting];
REVERT;

Warning: Null value is eliminated by an aggregate or other SET operation.

(18 rows affected)

Total execution time: 00:00:25.086

Route,Month,Schedules,Completed Trips,Bookings,Cancellations
APU - BTS,2022-06,40,40,0,0
APU - BTS,2022-07,20,0,0,0
APU - BTS,2022-09,20,0,0,0
APU - Bukit Jalil,2022-06,40,40,49,1
APU - Bukit Jalil,2022-07,20,0,0,0
APU - Bukit Jalil,2022-09,20,0,0,0
APU - Serdang KTM,2022-06,40,40,0,0
APU - Serdang KTM,2022-07,20,0,0,0
APU - Serdang KTM,2022-09,20,0,0,0
BTS - APU,2022-06,40,40,0,0


In [119]:
EXECUTE AS USER = 'AnalystDeptHead';
SELECT * FROM Management.[Weekly Reporting];
REVERT;

Warning: Null value is eliminated by an aggregate or other SET operation.

(24 rows affected)

Total execution time: 00:00:00.023

Route,Year,Week,Schedules,Completed Trips,Bookings,Cancellations
APU - BTS,2022,Week 22,20,20,0,0
APU - BTS,2022,Week 23,20,20,0,0
APU - BTS,2022,Week 29,20,0,0,0
APU - BTS,2022,Week 35,20,0,0,0
APU - Bukit Jalil,2022,Week 22,20,20,49,1
APU - Bukit Jalil,2022,Week 23,20,20,0,0
APU - Bukit Jalil,2022,Week 29,20,0,0,0
APU - Bukit Jalil,2022,Week 35,20,0,0,0
APU - Serdang KTM,2022,Week 22,20,20,0,0
APU - Serdang KTM,2022,Week 23,20,20,0,0


## Auditing

In [120]:
SELECT * FROM Management.AuditLog_Logon;
-- DELETE Management.AuditLog_Logon;

(14 rows affected)

Total execution time: 00:00:00.010

AuditLogID,LogDate,UserName,Spid
1,2022-07-13 02:51:20.510,MicrosoftAccount\manshun00@gmail.com,52
2,2022-07-13 02:51:20.827,MicrosoftAccount\manshun00@gmail.com,52
3,2022-07-13 02:51:20.890,MicrosoftAccount\manshun00@gmail.com,52
4,2022-07-13 02:51:21.470,MicrosoftAccount\manshun00@gmail.com,52
5,2022-07-13 02:51:24.020,MicrosoftAccount\manshun00@gmail.com,52
6,2022-07-13 02:51:28.063,MicrosoftAccount\manshun00@gmail.com,52
7,2022-07-13 02:51:28.293,MicrosoftAccount\manshun00@gmail.com,52
8,2022-07-13 02:51:28.763,MicrosoftAccount\manshun00@gmail.com,52
9,2022-07-13 02:51:30.850,MicrosoftAccount\manshun00@gmail.com,52
10,2022-07-13 02:51:32.117,MicrosoftAccount\manshun00@gmail.com,52


In [121]:
SELECT * FROM Management.AuditLog_DDL;

(19 rows affected)

Total execution time: 00:00:00.056

AuditLogID,LogDate,UserName,SQLCmd
1,2022-07-13 02:50:20.623,dbo,"CREATE TABLE Management.Station ( StationID INT PRIMARY KEY IDENTITY(1, 1), Name VARCHAR(200) UNIQUE NOT NULL, ShortName VARCHAR(20) UNIQUE NOT NULL )"
2,2022-07-13 02:50:21.350,dbo,"CREATE TABLE Management.Route ( RouteID INT PRIMARY KEY IDENTITY(1, 1), DepartureStationID INT REFERENCES Management.Station(StationID) NOT NULL, ArrivalStationID INT REFERENCES Management.Station(StationID) NOT NULL, Name VARCHAR(200) UNIQUE NOT NULL, CONSTRAINT Route_CK1 CHECK (DepartureStationID <> ArrivalStationID) )"
3,2022-07-13 02:50:22.170,dbo,"CREATE TABLE Management.TimeSlot ( TimeSlotID INT PRIMARY KEY IDENTITY(1, 1), DepartureDatetime DATETIME NOT NULL, ArrivalDatetime DATETIME NOT NULL, CONSTRAINT TimeSlot_CK1 CHECK (DepartureDatetime < ArrivalDatetime) )"
4,2022-07-13 02:50:24.393,dbo,"CREATE TABLE Management.BusStatus ( StatusID INT PRIMARY KEY IDENTITY(1, 1), Name VARCHAR(200) UNIQUE NOT NULL )"
5,2022-07-13 02:50:24.713,dbo,"CREATE TABLE Management.Bus ( BusID INT PRIMARY KEY IDENTITY(1, 1), PlateNumber VARCHAR(8) UNIQUE NOT NULL CHECK (DATALENGTH(PlateNumber) >= 7), Capacity INT NOT NULL CHECK (Capacity > 0), StatusID INT REFERENCES Management.BusStatus(StatusID) NOT NULL )"
6,2022-07-13 02:50:25.190,dbo,"CREATE TABLE Management.Schedule ( ScheduleID INT PRIMARY KEY IDENTITY(1, 1), RouteID INT REFERENCES Management.Route(RouteID) NOT NULL, TimeSlotID INT REFERENCES Management.TimeSlot(TimeSlotID) NOT NULL, BusID INT REFERENCES Management.Bus(BusID) NOT NULL, AvailableCapacity INT NOT NULL CHECK (AvailableCapacity >= 0) )"
7,2022-07-13 02:50:28.490,dbo,"CREATE TABLE BusUser.Student ( StudentID INT PRIMARY KEY IDENTITY(1, 1), Name VARCHAR(200) NOT NULL, UserName AS CAST(CONCAT('Student', StudentID) AS VARCHAR(200)) PERSISTED UNIQUE NOT NULL, Password VARCHAR(200) NOT NULL, EncryptedPasswordByKey VARBINARY(1000), EncryptedPasswordByCert VARBINARY(1000) )"
8,2022-07-13 02:50:28.770,dbo,"CREATE TABLE BusUser.Reservation ( ReservationID INT PRIMARY KEY IDENTITY(1, 1), StudentID INT REFERENCES BusUser.Student(StudentID) NOT NULL, ScheduleID INT REFERENCES Management.Schedule(ScheduleID) NOT NULL, ConfirmationNumber AS CAST(ReservationID + 100 AS INT) PERSISTED UNIQUE NOT NULL, Cancelled BIT NOT NULL DEFAULT 0, Datetime DATETIME NOT NULL DEFAULT GETDATE(), CONSTRAINT Reservation_AK1 UNIQUE (StudentID, ScheduleID) )"
11,2022-07-13 02:50:32.233,dbo,"CREATE OR ALTER VIEW BusUser.[Student Profile] WITH SCHEMABINDING AS SELECT StudentID AS [Student ID], Name, UserName, EncryptedPasswordByCert AS [Encrypted Password] FROM BusUser.Student;"
12,2022-07-13 02:50:32.527,dbo,"CREATE OR ALTER VIEW BusUser.[Bus Availability Checking] WITH SCHEMABINDING AS SELECT ScheduleID AS [Schedule ID], Route.Name AS Route, CAST(DepartureDatetime AS DATE) AS Date, CAST(DepartureDatetime AS TIME) AS [Departure Time], CAST(ArrivalDatetime AS TIME) AS [Arrival Time], PlateNumber AS [Plate Number], CONCAT(AvailableCapacity, '/', Capacity) AS Capacity FROM Management.Schedule LEFT JOIN Management.Route ON Schedule.RouteID = Route.RouteID LEFT JOIN Management.TimeSlot ON Schedule.TimeSlotID = TimeSlot.TimeSlotID LEFT JOIN Management.Bus ON Schedule.BusID = Bus.BusID WHERE DepartureDatetime BETWEEN GETDATE() AND DATEADD(day, 14, GETDATE()) ORDER BY DepartureDatetime, Route.RouteID OFFSET 0 ROWS;"


In [122]:
SELECT * FROM BusUser.AuditLog_Student;

(252 rows affected)

Total execution time: 00:00:00.069

AuditLogID,LogDate,AuditUserName,StudentID,Name,UserName,Password,UserAction
1,2022-07-13 02:50:48.260,dbo,50,Lulu,Student50,Passw0rd,INSERT
2,2022-07-13 02:50:48.260,dbo,49,Goibniu,Student49,Passw0rd,INSERT
3,2022-07-13 02:50:48.260,dbo,48,Yash,Student48,Passw0rd,INSERT
4,2022-07-13 02:50:48.260,dbo,47,Sipho,Student47,Passw0rd,INSERT
5,2022-07-13 02:50:48.260,dbo,46,Jaquan,Student46,Passw0rd,INSERT
6,2022-07-13 02:50:48.260,dbo,45,Cai,Student45,Passw0rd,INSERT
7,2022-07-13 02:50:48.260,dbo,44,Odeserundiye,Student44,Passw0rd,INSERT
8,2022-07-13 02:50:48.260,dbo,43,Ally,Student43,Passw0rd,INSERT
9,2022-07-13 02:50:48.260,dbo,42,Izabel,Student42,Passw0rd,INSERT
10,2022-07-13 02:50:48.260,dbo,41,Remedios,Student41,Passw0rd,INSERT


In [123]:
SELECT * FROM Management.AuditLog_Schedule;

(496 rows affected)

Total execution time: 00:00:00.045

AuditLogID,LogDate,UserName,ScheduleID,RouteID,TimeSlotID,BusID,AvailableCapacity,UserAction
1,2022-07-13 02:50:45.840,dbo,480,6,40,12,40,INSERT
2,2022-07-13 02:50:45.840,dbo,479,6,39,12,40,INSERT
3,2022-07-13 02:50:45.840,dbo,478,6,38,12,40,INSERT
4,2022-07-13 02:50:45.840,dbo,477,6,37,12,40,INSERT
5,2022-07-13 02:50:45.840,dbo,476,6,36,12,40,INSERT
6,2022-07-13 02:50:45.840,dbo,475,6,35,12,40,INSERT
7,2022-07-13 02:50:45.840,dbo,474,6,34,12,40,INSERT
8,2022-07-13 02:50:45.840,dbo,473,6,33,12,40,INSERT
9,2022-07-13 02:50:45.840,dbo,472,6,32,12,40,INSERT
10,2022-07-13 02:50:45.840,dbo,471,6,31,12,40,INSERT


In [124]:
SELECT * FROM Management.AuditLog_Route;

(6 rows affected)

Total execution time: 00:00:00.007

AuditLogID,LogDate,UserName,RouteID,DepartureStationID,ArrivalStationID,Name,UserAction
1,2022-07-13 02:50:43.317,dbo,6,4,1,BTS - APU,INSERT
2,2022-07-13 02:50:43.317,dbo,5,1,4,APU - BTS,INSERT
3,2022-07-13 02:50:43.317,dbo,4,3,1,Serdang KTM - APU,INSERT
4,2022-07-13 02:50:43.317,dbo,3,1,3,APU - Serdang KTM,INSERT
5,2022-07-13 02:50:43.317,dbo,2,2,1,Bukit Jalil - APU,INSERT
6,2022-07-13 02:50:43.317,dbo,1,1,2,APU - Bukit Jalil,INSERT


In [125]:
SELECT * FROM Management.AuditLog_TimeSlot;

(40 rows affected)

Total execution time: 00:00:00.008

AuditLogID,LogDate,UserName,TimeSlotID,DepartureDatetime,ArrivalDatetime,UserAction
1,2022-07-13 02:50:44.193,dbo,40,2022-09-03 21:00:00.000,2022-09-03 22:00:00.000,INSERT
2,2022-07-13 02:50:44.193,dbo,39,2022-09-03 20:00:00.000,2022-09-03 21:00:00.000,INSERT
3,2022-07-13 02:50:44.193,dbo,38,2022-09-03 18:00:00.000,2022-09-03 19:00:00.000,INSERT
4,2022-07-13 02:50:44.193,dbo,37,2022-09-03 16:00:00.000,2022-09-03 17:00:00.000,INSERT
5,2022-07-13 02:50:44.193,dbo,36,2022-09-03 14:00:00.000,2022-09-03 15:00:00.000,INSERT
6,2022-07-13 02:50:44.193,dbo,35,2022-09-03 11:00:00.000,2022-09-03 12:00:00.000,INSERT
7,2022-07-13 02:50:44.193,dbo,34,2022-09-03 10:00:00.000,2022-09-03 11:00:00.000,INSERT
8,2022-07-13 02:50:44.193,dbo,33,2022-09-03 09:00:00.000,2022-09-03 10:00:00.000,INSERT
9,2022-07-13 02:50:44.193,dbo,32,2022-09-03 07:00:00.000,2022-09-03 08:00:00.000,INSERT
10,2022-07-13 02:50:44.193,dbo,31,2022-09-03 06:00:00.000,2022-09-03 07:00:00.000,INSERT
